In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [ ]:
import numpy as np
from IPython.core.pylabtools import figsize

array = np.arange(24)
array2 = array.reshape(3,2,4)
print(array2)

In [ ]:
#print(array2[[1],:,1:3])
#print(array2[[2],[1],:])
print(array2.min(axis=0))

In [ ]:
import pandas as pd

In [ ]:
cars = pd.read_csv('cars.csv')
#cars.describe()
#cars.info()
cars.head(5)

In [ ]:
cars[['brand','owner']].head(5)

In [ ]:
def get_cars(city,owner):
    mask1 = cars['brand']==city
    mask2 = cars['owner']==owner
    return cars[mask1 & mask2].shape[0]
print(get_cars("Skoda","First Owner"))

In [ ]:
print(cars['owner'].value_counts())
t_car = cars[cars['owner']=="Fourth & Above Owner"]
print(t_car['brand'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
cars['brand'].value_counts().plot(kind='bar',figsize=(10,5))

In [ ]:
brand = cars.groupby('brand')

In [ ]:
brand.size().head(1)

In [ ]:
brand['km_driven'].mean().head(1)

In [ ]:
brand.get_group('BMW').head(1)

In [ ]:
cars.pivot_table(index='brand',columns = 'owner',values='km_driven',aggfunc='mean')
cars.head(1)

In [ ]:
import seaborn as sns
sns.boxplot(cars['selling_price'])

In [ ]:
##sns.boxplot(cars['selling_price'])
cars[cars['selling_price']==cars['selling_price'].max()]

In [ ]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
titanic = pd.read_csv('titanic_toy.csv')
titanic.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(titanic.drop(('Survived'),axis=1),titanic['Survived'], test_size=0.2, random_state=42)
scaler = StandardScaler()


In [ ]:
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [ ]:
x_test_scaled = pd.DataFrame(x_test_scaled,columns=x_test.columns)
x_test_scaled.head(10)

In [ ]:
titanic.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming Titanic dataset is loaded in a DataFrame called 'titanic'
sns.histplot(titanic['Age'], kde=True)  # For Age
plt.title('Age Distribution')
plt.show()


## ENCODING CATEGORICAL DATA
- nominal encoding -> this is when there's no relation between categories
- ordinal encoding -> when relationship between categories e.x. excellent, good
- label encoding -> ordinal encoding but specifically made for output

### ORDINAL ENCODING
- if we dont want to run some columns then we can slice the data and then merge later
- e.x. final_data = pd.concat([encoded_df, data['col3']], axis=1)






In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
oe = OrdinalEncoder(categories=[['Poor','Average','Good'],['School','UG','PG']])
x_train = oe.fit_transform(x_train)
x_test = oe.transform(x_test)
le = LabelEncoder()
# fit on y_train and y_test this does not need any input

### ONE HOT ENCODING
- pandas one hot encoding
- scikit learn one hot encoding
- k-1 one hot encoding
- one hot encoding with top categories -> when we too many categories

#### drop = 'first'
- this is done to avoid redundancy
- blue, green, red ball encoding problem
- avoid this for tree based models like xgboost, random forest, decision forest etc



In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
x_train_new = ohe.fit_transform(x_train[['fuel','owner']])
x_test_new = ohe.transform(x_test[['fuel','owner']])

## COLUMN TRANSFORMER
- tnf1 etc are just name give to find the columns if we want to see the pipelines



In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer # to fill in the missing values
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')
transformer.fit_transform(x_train)
transformer.transform(x_test)

## PIPELINES
- multiple steps chained together
- missing values impute -> encoding -> scaling -> feature selection -> decision tree model
- multiple columns transformers -> finally added to one pipeline
#### Pipeline VS make_pipeline
- pipelines requires naming of steps, make_pipeline does not

In [ ]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])
# where trf1 etc. are all column transformers, feature selection, model training
y_pred = pipe.predict(x_test)

## FUNCTION TRANSFORMERS
- implemented on columns to make data normal -> algorithm performance better
- log transform -> cannot be applied on -ve values
- reciprocal
- power
### POWER TRANSFORMERS
- Box-cox -> only +ve
- y(lambda) =
\begin{cases}
\frac{y^\lambda - 1}{\lambda}, & \text{if } \lambda \neq 0 \\
\ln(y), & \text{if } \lambda = 0
\end{cases}

- yeojohsnon -> modification of boxcox, applied to -ve also
- y(lambda) = \begin{cases}
\frac{(y + 1)^\lambda - 1}{\lambda}, & \text{if } \lambda \neq 0 \text{ and } y \geq 0 \\
\ln(y + 1), & \text{if } \lambda = 0 \text{ and } y \geq 0 \\
\frac{-( (-y + 1)^{2 - \lambda} - 1 )}{2 - \lambda}, & \text{if } \lambda \neq 2 \text{ and } y < 0 \\
-\ln(-y + 1), & \text{if } \lambda = 2 \text{ and } y < 0
\end{cases}


#### CHECK DATA
- use QQ plot
- if normally distributed, all points on the line of QQ plot
- if skewed right -> line going above ideal at end
- if skewed left -> line going below ideal at beginning

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
trf=FunctionTransformer(np.log1p) # logtransform we just log the values of column
# fit_transform this on x_train and transform x_test
# np.log1p adds 1 to value and then logs it, hence doesn't gives error if value - 0
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer() # by default works on yeo johnson just need to fit and transform

# BINNING
- grouping values in range
- helps in handling outliers
## UNSUPERVISED BINNING
### EQUAL WIDTH
- no change in the spread of data
- handles outliers

### EQUAL FREQUENCY
- handle outliers
- make data more uniform

### K BINNING

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
# add some other binner on other column and make a columntransformer

- see how to deal with mixed data, that is when data contains letter and number e.x. B53 PRANJWAL4798 etc
- see how to work with dates, python can convert columns to date type. written in notebook

## HANDLING MISSING DATA
### 1) REMOVING THE ROWS
- when data missing at random
- data missing <5% of actual data
### 2) IMPUTING THEM (FILLING)
- univariate(numerical, categorical)
- multivariate(KNN, iterative imputer)

#### Handling missing numerical data
- normal distribution -> mean/median
- skewed dis -> median
- randomly  -> useful with linear regression not with decision tree

#### Handling categorical missing data
- most frequent
- new missing
- random

In [ ]:
# df.isnull().mean()*100
from sklearn.impute import SimpleImputer
sip = SimpleImputer(strategy='median') # default is mean
# most_frequent etc

#categorical ->
df['categorical_col'].fillname('TA', inplace=True)
#or use a simple imputer with strategy = most_frequent

## KNNImputer
- more accurate, but has more calculations
- missing values imputed from mean from nearest neighbour (distance) found in training set

In [1]:
from sklearn.impute import KNNImputer

## MICE(multivariate imputation by chained equations)
- use when missing at random
- do not use when not missing at random
- iterative imputation
- check obsidian note

In [5]:
#from sklearn.impute import IterativeImputer
from sklearn.experimental import enable_iterative_imputer
#for mice

## OUTLIER DETECTION
- trimming (removing)
- capping (limiting)
- assuming them as missing values
- discretization

### For detecting outliers
- z score treatment -> works on col with almost normal distribution; z_score>3 -> outlier
- IQR based filtering -> boxplot Q1-1.5 * IQR , Q3+1.5 * IQR
- percentile -> 1 to 99 percentile boundary
- wiscorization


In [ ]:
#we can do capping using the np.where statement where we pass 3 parameters (condition,if true,if false)
# capping ->
df['cgpa'] = np.where(np['cgpa']>max_cgpa, max_cgpa,np.where(df['cgpa']<min_cgpa,min_cgpa,df['cgpa']))
#if greater than max_limit -> set to max_limit
#otherwise set to min_limit and if not keep as it is (for normal values)